In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from google.colab import userdata
import os

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


In [3]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

class ConversationAssistant:
    def __init__(self, llm_model) -> None:
        self.quant = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant = True,
            bnb_4bit_compute_dtype=torch.float16,
        )
        self.model = AutoModelForCausalLM.from_pretrained(llm_model, quantization_config=self.quant)
        self.tokenizer = AutoTokenizer.from_pretrained(llm_model)

    def generate_response(self, conversation):
        model_inputs = self.tokenizer.apply_chat_template(conversation, return_tensors="pt").to("cuda")
        generated_ids = self.model.generate(model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)[0]


In [4]:
    conversation = [
        {"role": "user", "content": "which is your favourite movie?"},
        {"role": "assistant", "content": "Well, I like twilight as it is a fictional movie and i find its character quite interesting"},
        {"role": "user", "content": "Do you have any other choice?"}
    ]

    llm_model = "mistralai/Mistral-7B-Instruct-v0.2"
    assistant = ConversationAssistant(llm_model)

    response = assistant.generate_response(conversation)
    print(response)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] which is your favourite movie? [/INST]Well, I like twilight as it is a fictional movie and i find its character quite interesting</s> [INST] Do you have any other choice? [/INST] Yes, I also enjoy other movies such as The Matrix, Inception, and Interstellar. All of these films offer mind-bending plots and thought-provoking concepts. The Matrix is a sci-fi action film with a deep philosophical underpinning. Inception is a complex and intriguing exploration of the human mind and the power of dreams. Interstellar is a science fiction epic about space travel and the importance of human connections in the face of great advers
